In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/07 14:46:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

In [6]:
df = spark.createDataFrame([
    Row(a=1, b=2, c='str1', d=date(2023, 1, 1), e=datetime(2023, 1, 1, 12, 0)),
    Row(a=2, b=3, c='str2', d=date(2023, 1, 2), e=datetime(2023, 1, 2, 12, 0)),
    Row(a=3, b=4, c='str3', d=date(2023, 1, 3), e=datetime(2023, 1, 3, 12, 0))]
)
df

DataFrame[a: bigint, b: bigint, c: string, d: date, e: timestamp]

In [7]:
df.show()

+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|  2|str1|2023-01-01|2023-01-01 12:00:00|
|  2|  3|str2|2023-01-02|2023-01-02 12:00:00|
|  3|  4|str3|2023-01-03|2023-01-03 12:00:00|
+---+---+----+----------+-------------------+



In [9]:
pandas_df = pd.DataFrame({
    'a':[1, 2, 3],
    'b':[2, 3, 4],
    'd':[date(2023, 1, 1), date(2023, 1, 2), date(2023, 1, 3)],
})
pandas_df.head()

,a,b,d
0,1,2,2023-01-01
1,2,3,2023-01-02
2,3,4,2023-01-03


In [10]:
spark_df = spark.createDataFrame(pandas_df)
spark_df.show()

+---+---+----------+
|  a|  b|         d|
+---+---+----------+
|  1|  2|2023-01-01|
|  2|  3|2023-01-02|
|  3|  4|2023-01-03|
+---+---+----------+



In [11]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [12]:
df.select('a', 'b', 'c').show()

+---+---+----+
|  a|  b|   c|
+---+---+----+
|  1|  2|str1|
|  2|  3|str2|
|  3|  4|str3|
+---+---+----+



In [14]:
df.select('a', 'b', 'c').describe().show()

+-------+---+---+----+
|summary|  a|  b|   c|
+-------+---+---+----+
|  count|  3|  3|   3|
|   mean|2.0|3.0|null|
| stddev|1.0|1.0|null|
|    min|  1|  2|str1|
|    max|  3|  4|str3|
+-------+---+---+----+



In [15]:
df.collect()

[Row(a=1, b=2, c='str1', d=datetime.date(2023, 1, 1), e=datetime.datetime(2023, 1, 1, 12, 0)),
 Row(a=2, b=3, c='str2', d=datetime.date(2023, 1, 2), e=datetime.datetime(2023, 1, 2, 12, 0)),
 Row(a=3, b=4, c='str3', d=datetime.date(2023, 1, 3), e=datetime.datetime(2023, 1, 3, 12, 0))]

In [16]:
df.toPandas()

/media/zmwaris1/mydrive/apps/Apache_Spark/spark-3.4.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,a,b,c,d,e
0,1,2,str1,2023-01-01,2023-01-01 12:00:00
1,2,3,str2,2023-01-02,2023-01-02 12:00:00
2,3,4,str3,2023-01-03,2023-01-03 12:00:00


In [19]:
df.select(df.a).show()

+---+
|  a|
+---+
|  1|
|  2|
|  3|
+---+



In [21]:
from pyspark.sql.functions import upper
df.withColumn('upper_c', upper(df.c)).show()

+---+---+----+----------+-------------------+-------+
|  a|  b|   c|         d|                  e|upper_c|
+---+---+----+----------+-------------------+-------+
|  1|  2|str1|2023-01-01|2023-01-01 12:00:00|   STR1|
|  2|  3|str2|2023-01-02|2023-01-02 12:00:00|   STR2|
|  3|  4|str3|2023-01-03|2023-01-03 12:00:00|   STR3|
+---+---+----+----------+-------------------+-------+



In [22]:
df.filter(df.a==1).show()

+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|  2|str1|2023-01-01|2023-01-01 12:00:00|
+---+---+----+----------+-------------------+



In [23]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50]
], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
+-----+------+---+---+



In [24]:
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    3.0|   30.0|
| blue|    3.0|   30.0|
+-----+-------+-------+



In [29]:
dest = spark.read.csv('file:///media/zmwaris1/mydrive/databases and datasets/packt/dest.csv', header=True)

In [30]:
dest.show()

+-----+---------+
|EmpNr|     Dest|
+-----+---------+
|    5|The Hague|
|    3|Amsterdam|
|    9|Rotterdam|
+-----+---------+



In [31]:
tips = spark.read.csv('file:///media/zmwaris1/mydrive/databases and datasets/packt/tips.csv', header=True)
tips.show()

+-----+------+
|EmpNr|Amount|
+-----+------+
|    5|    10|
|    9|     5|
|    7|   2.5|
+-----+------+



In [33]:
rdd1 = dest.rdd

[Row(EmpNr='5', Dest='The Hague'),
 Row(EmpNr='3', Dest='Amsterdam'),
 Row(EmpNr='9', Dest='Rotterdam')]

In [37]:
dest.join(tips, dest.EmpNr==tips.EmpNr, 'inner').show()

+-----+---------+-----+------+
|EmpNr|     Dest|EmpNr|Amount|
+-----+---------+-----+------+
|    5|The Hague|    5|    10|
|    9|Rotterdam|    9|     5|
+-----+---------+-----+------+



In [42]:
dest.join(tips, 'EmpNr', 'outer').show()

+-----+---------+------+
|EmpNr|     Dest|Amount|
+-----+---------+------+
|    3|Amsterdam|  null|
|    5|The Hague|    10|
|    7|     null|   2.5|
|    9|Rotterdam|     5|
+-----+---------+------+



In [44]:
dest.join(tips, 'EmpNr', 'left').show()

+-----+---------+------+
|EmpNr|     Dest|Amount|
+-----+---------+------+
|    5|The Hague|    10|
|    3|Amsterdam|  null|
|    9|Rotterdam|     5|
+-----+---------+------+



In [45]:
dest.join(tips, 'EmpNr', 'right').show()

+-----+---------+------+
|EmpNr|     Dest|Amount|
+-----+---------+------+
|    5|The Hague|    10|
|    9|Rotterdam|     5|
|    7|     null|   2.5|
+-----+---------+------+



In [48]:
df = spark.read.csv('file:///media/zmwaris1/mydrive/databases and datasets/packt/WHO_first9cols.csv', header=True)
df.show(5)

+-----------+---------+---------+-----------------------------+-----------------------+------------------------------------------------------+---------------------------------------------+-------------------------------------------+-------------------------------+
|    Country|CountryID|Continent|Adolescent fertility rate (%)|Adult literacy rate (%)|Gross national income per capita (PPP international $)|Net primary school enrolment ratio female (%)|Net primary school enrolment ratio male (%)|Population (in thousands) total|
+-----------+---------+---------+-----------------------------+-----------------------+------------------------------------------------------+---------------------------------------------+-------------------------------------------+-------------------------------+
|Afghanistan|        1|        1|                          151|                     28|                                                  null|                                         null|                 

In [52]:
country = df.select('Country')
country.show()
type(country)

+-------------------+
|            Country|
+-------------------+
|        Afghanistan|
|            Albania|
|            Algeria|
|            Andorra|
|             Angola|
|Antigua and Barbuda|
|          Argentina|
|            Armenia|
|          Australia|
|            Austria|
|         Azerbaijan|
|            Bahamas|
|            Bahrain|
|         Bangladesh|
|           Barbados|
|            Belarus|
|            Belgium|
|             Belize|
|              Benin|
|            Bermuda|
+-------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [54]:
df.columns

['Country',
 'CountryID',
 'Continent',
 'Adolescent fertility rate (%)',
 'Adult literacy rate (%)',
 'Gross national income per capita (PPP international $)',
 'Net primary school enrolment ratio female (%)',
 'Net primary school enrolment ratio male (%)',
 'Population (in thousands) total']

In [56]:
df.summary()

DataFrame[summary: string, Country: string, CountryID: string, Continent: string, Adolescent fertility rate (%): string, Adult literacy rate (%): string, Gross national income per capita (PPP international $): string, Net primary school enrolment ratio female (%): string, Net primary school enrolment ratio male (%): string, Population (in thousands) total: string]

In [57]:
df.toPandas().shape

(202, 9)

In [59]:
df.describe()

DataFrame[summary: string, Country: string, CountryID: string, Continent: string, Adolescent fertility rate (%): string, Adult literacy rate (%): string, Gross national income per capita (PPP international $): string, Net primary school enrolment ratio female (%): string, Net primary school enrolment ratio male (%): string, Population (in thousands) total: string]

In [60]:
df.count()

202

In [66]:
from pyspark.sql.functions import median, mean

In [68]:
df.stat.approxQuantile('CountryID', [0.5], 0.01)

IllegalArgumentException: requirement failed: Quantile calculation for column CountryID with data type StringType is not supported.